In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import bqplot as bq
import ipywidgets as widgets
import seaborn as sns

## First, lets price an easy derivative. Lets start with a future. A future is a contract for some asset (S) at some future time (T). Since we know how to move money into the future we can price a future contrat.  (assuming no dividends)
## $$F = S_t e^{r(T-t)}$$

In [ ]:
SP500 = 2850
T_t = 0.5 # 6months or half a year
r = 0.005 # Annualized rate of 0.5% 
F = SP500*np.exp(r*T_t)
print(F)

## The data shows asset prices are random from moment to moment, or at least can be modeled that way. 
## Lets look at a stochastic process 

## $$ \frac{dS}{S} = \mu  dt + \sigma dW$$



## The Value of an Option (V) is known at expiration (T) for any underlying price (S)
## $$ V(S,T)$$

## How does that value evolve over time? well by ito's lemma on the stochastic process:
## $$ dV = \left(\mu S \frac{\partial V}{\partial S} + \frac {\partial V}{\partial t} + \frac {1}{2} \sigma^2 S^2 \frac {\partial^2 V}{\partial S^2} \right) dt + \sigma S \frac{\partial V}{\partial S} dW$$

## By solving this for the specifc case of the delta hedged portfolio we arrive at 
## $$ \frac {\partial V}{\partial t} + \frac {1}{2} \sigma^2 S^2 \frac {\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0  $$

## if you take this to the math department and asked them to solve this PDE for the value of a call option they will bring you back the Black Scholes Merton model. 
## $$C(S_t,t)  = N(d_1)S_t - N(d_2)PV(K)  $$

## $N()$ is the cumulative distribution function for a standard normal distribution  and $PV(K)$ is the present value of the strike price. 

## $$d_1 = \frac{ln(\frac{S_t}{K}) + \left(r + \frac{\sigma^2}{2}\right) (T-t)}{\sigma \sqrt{T-t}}$$

## $$d_2 = d_1 - \sigma \sqrt{T-t}$$

## ok, finally lets play:

In [ ]:
#Set up the values with what we know

St =100
K = 120 # K is the strike price
T_t = 1 #1.5 # T-t in years, so 1 = one year
Vol = 0.15
r = 0.0014 # the risk free interest rate (annualized)


In [ ]:
d1 = (np.log(St/K)+(r+0.5*Vol**2)*T_t)/(Vol*np.sqrt(T_t))
d2 = d1 - (Vol*np.sqrt(T_t))
PV_K  = K*np.exp(-r*T_t)

Call_price = norm.cdf(d1)*St - norm.cdf(d2)*PV_K

In [ ]:
Call_price

In [ ]:
def VestaSim(I):
    
    S = St * np.exp(np.cumsum((r - 0.5 * Vol ** 2) * dt
                + Vol * np.sqrt(dt) * np.random.standard_normal((M + 1, I)), axis=0))
    S[0] = St
    return S

In [ ]:
T = T_t; M = 12*T_t; dt = T / M   
numSims = 50000 
MC = VestaSim(numSims)

In [ ]:
#Setting up the graph parameters
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

line = bq.Lines(x=np.arange(len(MC)), y=MC.transpose(),
             scales={'x': sc_x, 'y': sc_y})
ax_x = bq.Axis(scale=sc_x, label='months')
ax_y = bq.Axis(scale=sc_y, orientation='vertical', label='Values')

Rando = bq.Figure(marks=[line], axes=[ax_x, ax_y], title='Random Values')

In [ ]:
#Display the Graph
Rando

If we aren't graphing the paths we can solve the end values in a different way. *******

In [ ]:
def generate_asset_price(St,Vol,r,T_t):
    return St * np.exp((r - 0.5 * Vol**2) * T_t + Vol * np.sqrt(T_t) * np.random.standard_normal())
def call_payoff(S_T,K):
    return max(0.0,S_T-K)

In [ ]:
payoffs =[]
for i in range(numSims):
    S_T = generate_asset_price(St,Vol,r,T_t)
    payoffs.append(
        call_payoff(S_T, K)
    )

**************

In [ ]:
DataFrame = pd.DataFrame(MC) # put payoffs here instead of MC if you use the other method. 

In [ ]:
DataFrame.iloc[M-1].describe()

In [ ]:
display(sns.distplot(DataFrame.iloc[M-1]))

In [ ]:
Payoffs = DataFrame.iloc[M-1]-K


In [ ]:
Payoffs[Payoffs <= 0] = 0

In [ ]:
ExpectedPayoff = Payoffs.mean()

In [ ]:
ExpectedPayoff*np.exp(-r*T_t)

## lets look at the distribution of possible PnL for the asset then the call option. 

In [ ]:
display(sns.distplot(DataFrame.iloc[M-1]-St))

In [ ]:
display(sns.distplot(Payoffs)) #Payoffs[Payoffs > 0]

In [ ]:
display(sns.distplot(Payoffs),sns.distplot(DataFrame.iloc[M-1]-St))